# Exploratory data analysis

In [1]:
from parse_data import TextsinlevelsDB

In [2]:
textsinlevels = TextsinlevelsDB(db_name="textsinlevels")
news = textsinlevels.write_from_table_to_df("newsinlevels")
days = textsinlevels.write_from_table_to_df("daysinlevels")
del textsinlevels

In [3]:
def print_general_info(df):
    print(df.head())
    print()
    print("The number of texts:", df.shape[0])
    print(f"The number of texts for each level:\n{df['level'].value_counts().sort_index()}")

In [4]:
print_general_info(news)

               date                           heading  \
0  10-01-2022 07:00            Famous couples in 2021   
1  08-01-2022 12:00         How to Use News in Levels   
2  07-01-2022 15:00  Fire at South African parliament   
3  07-01-2022 07:00               Israel attacks Gaza   
4  06-01-2022 15:00            Man gives food to dogs   

                                        article_text  level  
0  Many famous couples break up last year. Many f...      1  
1  News in Levels helps students learn English. D...      1  
2  A fire starts in the parliament building in Ca...      1  
3  Hamas is a militant group in Gaza. It fights w...      1  
4  A man from Jordan loves animals a lot. He help...      1  

The number of texts: 9798
The number of texts for each level:
1    3266
2    3266
3    3266
Name: level, dtype: int64


In [5]:
print_general_info(days)

         date                                            heading  \
0  09-01-1959        The day Fidel Castro took control over Cuba   
1  05-01-1933  The day the Golden Gate Bridge construction st...   
2  26-12-2004         The day of the tsunami in the Indian Ocean   
3  25-12-1979            The day the Soviets invaded Afghanistan   
4  24-12-1914               The day the enemies met at Christmas   

                                        article_text  level  
0  It is 1956. Fidel Castro wants a revolution in...      1  
1  The Golden Gate Bridge is a symbol of San Fran...      1  
2  It is Sunday morning. It is December 26, 2004....      1  
3  It is December 25, 1979. Three hundred planes ...      1  
4  In 1914, World War I starts in Europe. The ene...      1  

The number of texts: 468
The number of texts for each level:
1    156
2    156
3    156
Name: level, dtype: int64
